# Analyse de tweets

In [ ]:
import os
import json
import collections

In [ ]:
print(os.getcwd())

In [ ]:
## pour se mettre dans le bon répértoire 
os.chdir("../../../../../Desktop/twitter")
print(os.getcwd())

Vous pouvez commencer par ouvrir le fichier 'tweet1.json' avec un éditeur de texte tel que Notepad++, puis avec Firefox. Ce fichier contient les informations relatives à 1 tweet. Parvenez-vous à répondre aux questions suivante: 
* Quel est le texte du tweet ?
* Quand a été émis ce tweet ?
* Qui est l'auteur de ce tweet ?
* Ce tweet est-il un retweet?

On va maintenant répondre à ces questions dans python:

In [ ]:
fichier = "tweet1.json"
with open(fichier) as f:
    for line in f:
        tweet = json.loads(line)
        print("texte:", tweet["text"])
        print("Date:", tweet["created_at"])
        print("Auteur-ice:", tweet["user"]["name"])
        print("Retweet: ","retweeted_status" in tweet)

On va maintenant examiner le message orginal qui a été retweeté et qui contenait une citation.
* Qui l'a écrit ? 
* Quand a-t-il été écrit ?
* Quelle est cette citation ?
* Qui l'a-t-il écrite ? 

In [ ]:
fichier = "tweet1.json"
with open(fichier) as f:
    for line in f:
        tweet = json.loads(line)
        print("Auteur-ice:", tweet["retweeted_status"]["user"]["name"])
        print("Date:", tweet["retweeted_status"]["created_at"])
        print("Citation:", tweet["retweeted_status"]["quoted_status"]["text"])
        print("Par:", tweet["retweeted_status"]["quoted_status"]["user"]["name"])
        print("Le: ", tweet["retweeted_status"]["quoted_status"]["created_at"])

On va maintenant regarder des tweets qui ont tous été écrits pendant la même minute et qui se trouvent dans le fichier 'minute.json' que vous pouvez ouvrir avec votre éditeur de texte. 
* Quand ces tweets ont-il été écrits ?
* Combient y en a-t-il ? 

In [ ]:
fichier = "minute.json"
cnt = 0
with open(fichier) as f:
    for line in f:
        try:
            tweet = json.loads(line)
        except Exception:
            print(line)
        cnt = cnt+1
print (cnt, "tweets, dont le dernier écrit le", tweet["created_at"])

### Filtrage de tweets

On va maintenant afficher tous les tweets qui mentionnent "Biden". Pour ceci on va passer en revue et traiter tous les tweets
du fichier

In [ ]:
fichier = "minute.json" ## le nom du fichier
cnt = 0
def traitement(tweet): ## le traitement de chaque tweet
    global cnt
    if "text" in tweet:
        if "Biden" in tweet["text"]:
            print(tweet["text"])
            cnt = cnt+1

## on applique le traitement à chaque tweet du fichier
with open (fichier) as f:
    for line in f:
        tweet = json.loads(line)
        traitement(tweet)
print("Il y a ",cnt, "tweets") 

On va maintenant voir examiner les tweet qui ont été écris le 5 novembre 2020 entre 18 et 21 (heure américaine) et afficher ceux qui mentionnent Biden.

In [ ]:
cnt= 0
interruption = False
datadir = "./data/2020"

def traitement(tweet): ## le traitement de chaque tweet
    global cnt
    if "text" in tweet:
        if "Biden" in tweet["text"]:
            print(tweet["text"])
            cnt = cnt+1

def parcours_tweet(jours, heures):
    global interruption
    interruption = False
    for day in jours:
        for hour in heures:
            for minute in range(60): # pour les minutes 0 à 59
                fichier = datadir + "11/" + format(day,'02d')+ "/" + format(hour,'02d') + "/" + format(minute,'02d')+".json"
                if os.path.isfile(fichier):
                    with open (fichier) as f:
                        for line in f:
                            tweet = json.loads(line)
                            traitement(tweet)
                            if interruption: 
                                return
                else:
                    print(fichier,"introuvable")
    print("\n\n ----- Tous les fichiers ont été parcourus -----")
    return
parcours_tweet(range(5,6), range(18,21)) ## parcourir tous les tweets du 4 novembre entre 18h et 21h
print("Il y a ",cnt, "tweets") 

On va maintentant afficher les tweets écrits en français entre le 4 novembre entre 18h et 19h:

In [ ]:
cnt= 0

def traitement(tweet): ## le traitement de chaque tweet
    global cnt
    if 'lang' in tweet:
        if tweet['lang'] == 'fr': ## la langue est le français
            print(tweet['text'])
            cnt = cnt + 1

parcours_tweet(range(5,6), range(18,19))
print("Il y a ", cnt, 'tweets.')

Vous pouvez modifier le code ci-dessus pour faire d'autres recherche...

### Opinion politique et niveau socio-économique
On va maintenant voir si ces données peuvent nous indiquer des différences statistiques entre les personnes mentionnant Trump et celles mentionnant Biden. Comme les iPhones sont en général plus chers que les téléphones Android, la proportion de tweets émis par des iPhones peut nous donner une indication sur le niveau socio-économique des auteurs d'une certain classe de tweet. On va donc voir si les tweet mentionnant Biden sont plus souvent émis à partir d'iPhone que les tweet mentionnant Trump. Cela pourrait être le cas, si Biden a plus de soutien dans les classes plus favorisées de la société.

In [ ]:
# initialisation des compteurs a 0
biden_ios = 0
biden_android = 0
trump_ios = 0
trump_android = 0

def traitement(tweet): ## le traitement de chaque tweet
    global biden_ios, biden_android, trump_ios, trump_android
    if 'text' in tweet and 'source' in tweet:
        if "Biden" in tweet['text'] and not "Trump" in tweet['text']: ## mentionne Biden mais pas Trump
            if 'iPhone' in tweet['source']:
                biden_ios += 1
            if 'Android' in tweet['source']:
                biden_android += 1
        if not "Biden" in tweet['text'] and "Trump" in tweet['text']: ## mentionne Trump mais pas Biden
            if 'iPhone' in tweet['source']:
                trump_ios += 1
            if 'Android' in tweet['source']:
                trump_android += 1


parcours_tweet(range(4,5), range(18,19))
trump_ios_ratio = trump_ios/(trump_ios+trump_android)
biden_ios_ratio = biden_ios/(biden_ios+biden_android)
print("Trump: {:.2%} des tweets le mentionnant proviennent d'un iPhone".format(trump_ios_ratio))
print("Biden: {:.2%} des tweets le mentionnant proviennent d'un iPhone".format(biden_ios_ratio))

Que pensez-vous de ce résultat? Est-ce que cela vous paraît concluant? Quelles sont les limites de cette analyse? 

### Popularité de tweets
On va regarder les retweets les plus fréquents pendant cette heure

In [ ]:
cnt= 0
idcount = collections.defaultdict(int)

def traitement(tweet): ## le traitement de chaque tweet
    if "retweeted_status" in tweet:
        if tweet['retweeted_status']['retweet_count'] > 10000:
            if 'id' in tweet['retweeted_status']:
                idcount[tweet['retweeted_status']['id']] += 1

parcours_tweet(range(4,5), range(18,19))
vals = sorted(list(idcount.values()))
retweetid = [k for k in idcount.keys() if idcount[k] in  vals[-10:]]
print(retweetid)

On va maintenant regarder à quoi correspondent ces tweets: 

In [ ]:
dejavu = set()

def traitement(tweet): ## le traitement de chaque tweet
    global interruption
    if "retweeted_status" in tweet:
        if 'id' in tweet['retweeted_status']:
            if tweet['retweeted_status']['id'] in retweetid:
                if tweet['retweeted_status']['id'] not in dejavu:
                    print(tweet['retweeted_status']['id'], ":", tweet['retweeted_status']['text'] )
                    dejavu.add(tweet['retweeted_status']['id'])
                    if len(dejavu) == len(retweetid):
                        interruption = True
                    
parcours_tweet(range(4,5), range(18,19))

On va maintenant prendre les retweet les plus fréquents sur le 4 et 5 novembre et compter pour chaque heure combien de fois ils ont été retweetés. Puis on va en faire un graphique.

In [ ]:
datadir ='../data/2020d/'
timeline = {}
timeidx = 0
shortlist = [1323857811734933504, 1323879858485821440, 1324003503526694914, 1324125650630766592, 1324198817076441089, 1324251847478173701, 1324365921452912640, 1324372378672156674, 1324347819281534979, 1324439705522524162]
texts = {}
for idx in shortlist:
    timeline[idx] = []
    
for day in range(4,6):
    for hour in range(24):
        for idx in shortlist:
                timeline[idx].append(0)
        for minute in range(0,60,3): #pour gagner du temps, on ne prend qu'une minute sur trois.
            fname = datadir+ "11/" + format(day,'02d')+ "/" + format(hour,'02d') + "/" + format(minute,'02d')+".json"
            print(fname, end="")
            if os.path.isfile(fname):
                print(" - open")
                with open (fname) as f:
                    for line in f:
                        tweet = json.loads(line)
                        if 'retweeted_status' in tweet:
                                if 'id' in tweet['retweeted_status']:
                                    if tweet['retweeted_status']['id'] in shortlist:
                                        timeline[tweet['retweeted_status']['id']][timeidx] += 1
                                        if tweet['retweeted_status']['id'] not in texts.keys():
                                            texts[tweet['retweeted_status']['id']] = tweet['retweeted_status']['text'] + ' - ' + tweet['retweeted_status']['user']['name']
            else:
                print("")
        timeidx += 1

Maintenant qu'on a compté les retweet, on va dessiner le graphique et montrer le texte et les auteurs ou autrices des tweets.

In [ ]:
import matplotlib.pyplot as plt

for k in timeline.keys():
    plt.plot(timeline[k], label=texts[k])
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.ylim([0, 30])
plt.xlabel("Heures")
plt.ylabel("Nombre de retweets")
plt.show()
for k in texts:
    print(texts[k])

Quels sont les sujets qui sont les plus populaires dans ce jeu de données? 